In [ ]:
#| default_exp fullfactorial

# FullFactorial

In [ ]:
#| export
import numpy as np
import pandas as pd

from numpy.random import default_rng

from fastDOE.core import ExperimentalDesign

In [ ]:
#| hide
import fastcore.test as test

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export

def _generate_levels(factorlvls: list) -> list:
    """Generates a list of the levels from
    the given list with numbers of levels for each factor.

    [2, 3, 2] -> [[0, 1], [0, 1, 2], [0, 1]]

    Args:
        factorlvls (list): list with the number of levels for each factor

    Returns:
        list: list with the levels for each factor
    """
    levels = []
    for rnge in factorlvls:
        assert(type(rnge) == int)
        levels.append(list(range(rnge)))
    return levels

In [ ]:
nr_lvls = [2, 3, 4]

_generate_levels(nr_lvls)

[[0, 1], [0, 1, 2], [0, 1, 2, 3]]

In [ ]:
#| export

def _build_ff_column(factor:int, nr_levels: list, levels: list) -> list:
    """Builds the experimental column for the given factor.

    Args:
        col (int): the column to build
        nr_levels (list): number of levels per factor e.g. [2, 3, 4]
        levels (list): level for each factor e.g. [[0, 1], [0, 1, 2, 3], [0, 1, 2, 3, 4]]

    Returns:
        list: column of the experimental design matrix for the given factor
    """
    lvls = [1, 1] + nr_levels
    n = np.array(nr_levels).prod()
    reps = lvls[factor] * lvls[factor +1]
    unit = []
    for i in levels[factor]:
        unit.extend([i] * reps)
    column = unit * int(n / len(unit))
    return column
   
    

In [ ]:
#| export
def fullfactorial(nr_lvls: list = [], levels: list = []) -> np.ndarray:
    """Generates a general fullfactorial experimental design matrix

    Args:
        factorlvls (list): _description_ e.g. [2, 3, 3]. Defaults to [].
        levels (list, optional): _description_. eg. [[1, 2], [1, 2, 3], [1, 2, 3]] Defaults to [].

    Returns:
        np.ndarray: _description_
    """
    if not nr_lvls and not levels:
        raise ValueError("Please provide either nr_lvls or levels.")
    nr_lvls = [len(x) for x in levels] if not nr_lvls else nr_lvls
    levels = _generate_levels(nr_lvls) if not levels else levels

    # build the design matrix
    matrix = []
    for i, lvl in enumerate(nr_lvls):
        matrix.append(_build_ff_column(i, nr_lvls, levels))
    matrix = np.array(matrix)
    return matrix.T
    

In [ ]:
ff = fullfactorial(nr_lvls)
ff

array([[0, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 1, 0],
       [0, 2, 0],
       [1, 2, 0],
       [0, 0, 1],
       [1, 0, 1],
       [0, 1, 1],
       [1, 1, 1],
       [0, 2, 1],
       [1, 2, 1],
       [0, 0, 2],
       [1, 0, 2],
       [0, 1, 2],
       [1, 1, 2],
       [0, 2, 2],
       [1, 2, 2],
       [0, 0, 3],
       [1, 0, 3],
       [0, 1, 3],
       [1, 1, 3],
       [0, 2, 3],
       [1, 2, 3]])

In [ ]:
#| export

def _build_ff2k_column(
    col: int, # column to build
    n: int # number of runs
) -> np.ndarray:
    repeats = 2**col
    entry = 1
    column = []
    for i in range(n):
        if i % repeats == 0:
            entry *= -1
        column.append(entry)
    return np.array(column).reshape(-1, 1)

In [ ]:
#| hide
test.array_equal(_build_ff2k_column(0, 8), np.array([-1, 1, -1, 1, -1, 1, -1, 1]).reshape(-1, 1))
test.array_equal(_build_ff2k_column(1, 8), np.array([-1, -1, 1, 1, -1, -1, 1, 1]).reshape(-1, 1))
test.array_equal(_build_ff2k_column(2, 8), np.array([-1, -1, -1, -1, 1, 1, 1, 1]).reshape(-1, 1))

True

In [ ]:
#| hide
test.array_equal(
    _build_ff2k_column(2, 16),
    np.array([-1, -1, -1, -1, 1, 1, 1, 1, -1, -1, -1, -1, 1, 1, 1, 1]).reshape(-1, 1)
)
test.array_equal(
    _build_ff2k_column(3, 16),
    np.array([-1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1]).reshape(-1, 1)
)

True

In [ ]:
#| export
def fullfactorial2k(k: int) -> np.ndarray:
    """
    Creates a full factorial design with 2 levels per factor
    in standard order

    if you have 5 or more factors consider screening designs
    to reduce the number of factors

    k (int): number of factors

    Return
    """
    if k < 1:
        raise ValueError("number of factors has to be greater than 1")
    nr_lvls = [2] * k
    n = 2**k
    matrix = np.concatenate([_build_ff2k_column(col, n) for col in range(k)], axis=1)
    return matrix


full factorial design matrix with 3 factors

In [ ]:
fullfactorial2k(3)

array([[-1, -1, -1],
       [ 1, -1, -1],
       [-1,  1, -1],
       [ 1,  1, -1],
       [-1, -1,  1],
       [ 1, -1,  1],
       [-1,  1,  1],
       [ 1,  1,  1]])

In [ ]:
fullfactorial2k(4)

array([[-1, -1, -1, -1],
       [ 1, -1, -1, -1],
       [-1,  1, -1, -1],
       [ 1,  1, -1, -1],
       [-1, -1,  1, -1],
       [ 1, -1,  1, -1],
       [-1,  1,  1, -1],
       [ 1,  1,  1, -1],
       [-1, -1, -1,  1],
       [ 1, -1, -1,  1],
       [-1,  1, -1,  1],
       [ 1,  1, -1,  1],
       [-1, -1,  1,  1],
       [ 1, -1,  1,  1],
       [-1,  1,  1,  1],
       [ 1,  1,  1,  1]])

In [ ]:
ff = fullfactorial2k(5)
assert ff.shape[0] == 2**5
assert ff.shape[1] == 5

In [ ]:
#| export
def randomize_frame(df: pd.DataFrame, randomize=True, axis: int=0, seed=42) -> pd.DataFrame:
    """Randomizes the rows of the Dataframe
    if randomize is True.

    Args:
        df (pd.DataFrame): _description_
        randomize (bool, optional): _description_. Defaults to True.
        axis (int, optional): _description_. Defaults to 0.
        seed (int, optional): _description_. Defaults to 42.

    Returns:
        pd.DataFrame: _description_
    """
    if randomize:
        rng = default_rng(seed)
        rng.shuffle(df.values, axis=axis)
    return df

In [ ]:
#| export
def add_blocks(df: pd.DataFrame, block_on:str) -> pd.DataFrame:
    """Adds blocking to the given Dataframe
    for the given columns to block_on.

    Args:
        df (pd.DataFrame): _description_
        block_on (str): column which is used to block the experimental Design

    Returns:
        pd.DataFrame: _description_
    """
    i = 0
    if block_on == "rep":
        for r in df["rep"].unique():
            df.loc[(df["rep"]==r), "block"] = i
            i += 1
    elif block_on in df.columns:
        for r in df["rep"].unique():
            for v in df[block_on].unique():
                df.loc[(df["rep"]==r) & (df[block_on]==v), "block"] = i
                i+=1
    return df


In [ ]:
#| export
def add_centerruns(matrix: pd.DataFrame, centerruns) -> pd.DataFrame:
    """Adds centerruns to  the given design matrix

    Args:
        matrix (pd.DataFrame): _description_
        centerruns (str | int): either "block" to add centerruns after each block
                                or specify the number of centerruns directly which
                                are distributed evenly across the design matrix

    Returns:
        pd.DataFrame: _description_
    """
    n = matrix.shape[0]
    if centerruns == "block":
        centerruns = len(matrix["block"].unique()) + 1
    elif type(centerruns) == int:
        centerruns = centerruns
    centerpositions = np.linspace(0, n, centerruns, dtype="int")
    new_matrix = np.insert(matrix.values, centerpositions, [0]*matrix.shape[1], axis=0)
    return pd.DataFrame(new_matrix, columns=matrix.columns)


In [ ]:
#| export
def fullfact2k(
    factors:list, # list of experimental factors
    center_vals:list=[], # list of center values for each factor
    delta_vals: list= [], # list of delta values for each factor
    randomize:bool=True, # randomize the design
    centerruns:int=3, # number of centerruns
    repetitions:int=0, # number of repetitions
    block_on: int="rep", # factor which is used to block the design
    seed:int=42 # seed for the randomization
):
    """Wrapper for fullfactorial2k with additional options

    Args:
        factors (list): _description_
        randomize (bool, optional): _description_. Defaults to True.
        centerruns (int, optional): _description_. Defaults to 3.
        repetitions (int, optional): _description_. Defaults to 0.
        seed (int, optional): _description_. Defaults to 42.

    Returns:
        _type_: _description_
    """
    
    k: int = len(factors)
    runs: int = repetitions + 1
    n: int = 2**k * runs
    model_matrix: np.ndarray = fullfactorial2k(k)
    start: int = 1
    stop: int = len(model_matrix)
    steps: int = len(model_matrix)
    matrix = np.append(model_matrix, np.linspace(start, stop, steps, dtype="int32").reshape((steps, -1)), axis=1)
    cols = factors + ["standardOrder"]
    df = pd.DataFrame(matrix, columns = cols)
    new_df = df.assign(rep=0)
    for i in range(1, runs):
        new_df = pd.concat([new_df, df.assign(rep=i)])
    design = (
        new_df
        .assign(block=0)
        .pipe(add_blocks, block_on)
        .pipe(randomize_frame, randomize)
        .sort_values("rep")
        .pipe(add_centerruns, centerruns)
    )
    return ExperimentalDesign(design, factors, center_vals=center_vals, delta_vals=delta_vals)

In [ ]:
factors = ["a", "b", "c"]
design = fullfact2k(factors, block_on="rep", repetitions=1); design

    a  b  c  standardOrder  rep  block
0   0  0  0              0    0      0
1  -1  1  1              7    0      0
2   1  1 -1              4    0      0
3  -1 -1 -1              1    0      0
4   1  1  1              8    0      0
5   1 -1  1              6    0      0
6  -1  1 -1              3    0      0
7  -1 -1  1              5    0      0
8   1 -1 -1              2    0      0
9   0  0  0              0    0      0
10  1  1  1              8    1      1
11  1  1 -1              4    1      1
12 -1  1 -1              3    1      1
13  1 -1 -1              2    1      1
14 -1 -1  1              5    1      1
15 -1  1  1              7    1      1
16  1 -1  1              6    1      1
17 -1 -1 -1              1    1      1
18  0  0  0              0    0      0

In [ ]:
#| hide
from nbdev import nbdev_export; nbdev_export()